In [88]:
from collections import defaultdict
import nltk
from string import punctuation
nltk.download('gutenberg')
nltk.download('punkt')

from nltk.corpus import gutenberg
from nltk import word_tokenize

class BPETokenizer:
    def __init__(self, text, num_merges):
        self.token_vocab = set()
        self.corpus = {}
        self.num_merges = num_merges
        self.merges = set()
        self.text = self.preprocess_text(text)
        
    def preprocess_text(self, text):
        text = text.replace("\n", " ")
        # clean_text = ''.join(char for char in text if char not in punctuation)
        clean_text = text
        clean_text = ' '.join(clean_text.split())
        return clean_text

    def get_vocab(self):
        vocab = set(char for word in self.text.split() for char in word)
        vocab.add("_")
        return vocab

    def count_words(self):
        freqs = defaultdict(int)
        for word in self.text.split():
            word += "_"
            freqs[word] += 1
        return freqs

    def find_pairs(self):
        pairs = defaultdict(int)
        
        for word, count in self.corpus.items():
            tokens = word.split()
            if len(tokens) < 2:
                continue
            for i in range(len(tokens) - 1):
                pair = (tokens[i], tokens[i + 1])
                pairs[pair] += count
                
        return pairs

    def merge_pair(self, pair):
        new_dict = defaultdict(int)
        bigram = " ".join(pair)
        merged = "".join(pair)
        
        for word, count in self.corpus.items():
            new_word = word.replace(bigram, merged)
            new_dict[new_word] = count
            
        return new_dict
    
    
    def BPE(self):
        self.token_vocab = self.get_vocab()
        
        word_freqs = self.count_words()
        self.corpus = {' '.join(word): freq for word, freq in word_freqs.items()}
        
        for _ in range(self.num_merges):
            pair_freqs = self.find_pairs()
            if not pair_freqs:
                break

            best_pair = max(pair_freqs.items(), key=lambda x: x[1])[0]
            self.merges.add(best_pair)
            self.token_vocab.add(''.join(best_pair))
            self.corpus = self.merge_pair(best_pair)
            
        return self.corpus, self.token_vocab, self.merges
    
    def encode(self, text):
        processed_text = self.preprocess_text(text)
        result = []
        
        for word in processed_text.split():
            chars = []
            current_token = ""
            
            for char in word:
                if char.isalnum() or char == '_':
                    current_token += char
                else:
                    if current_token:
                        chars.append(current_token)
                        current_token = ""
                    chars.append(char)
            
            if current_token != "":
                chars.append(current_token)
            
            for token in chars:
                token_with_suffix = f"{token}_"
                
                if token_with_suffix in self.corpus:
                    result.append(token_with_suffix)
                    continue
                    
                current = ' '.join(token_with_suffix)
                
                while True:
                    performed_merge = False
                    
                    for merge_pair in self.merges:
                        merge_str = ' '.join(merge_pair)
                        if merge_str in current:
                            merged = ''.join(merge_pair)
                            current = current.replace(merge_str, merged)
                            performed_merge = True
                            
                    if not performed_merge:
                        break
                        
                result.extend(current.split())
        
        return result
    
    def decode(self, encoded_text):
        return ' '.join(self.token_vocab[i] for i in encoded_text)
    

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/dhruvgorasiya/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dhruvgorasiya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:


if __name__ == "__main__":
    book1 = gutenberg.raw("austen-emma.txt")
    book2 = gutenberg.raw("blake-poems.txt") 
    book3 = gutenberg.raw("shakespeare-hamlet.txt")
    
    training_text = book1 + " " + book2 + " " + book3
    
    test_book1 = gutenberg.raw("shakespeare-caesar.txt")
    test_book2 = gutenberg.raw("carroll-alice.txt")
    test_book3 = gutenberg.raw("chesterton-ball.txt")
    
    reference_tokenizations = {
        'shakespeare-caesar': word_tokenize(test_book1),
        'carroll-alice': word_tokenize(test_book2),
        'chesterton-ball': word_tokenize(test_book3)
    }
        
    num_merges = 100000
    bpe_tokenizer = BPETokenizer(training_text, num_merges)
    corpus, vocab, merges = bpe_tokenizer.BPE()
    
    def calculate_metrics(reference_tokens, bpe_tokens):
        ref_vocab = set(reference_tokens)
        bpe_vocab = set(bpe_tokens)
        
        true_positives = len(ref_vocab.intersection(bpe_vocab))
        false_positives = len(bpe_vocab - ref_vocab) 
        false_negatives = len(ref_vocab - bpe_vocab)
        
        precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        jaccard_similarity = len(ref_vocab.intersection(bpe_vocab)) / len(ref_vocab.union(bpe_vocab)) if ref_vocab or bpe_vocab else 0

        tokenization_accuracy = ((true_positives / len(bpe_tokens)) * 100) if len(bpe_tokens) > 0 else 0
        tokenization_coverage = ((len(bpe_vocab) / len(ref_vocab)) * 100) if len(ref_vocab) > 0 else 0
        
        metrics = {
            'ref_vocab_size': len(ref_vocab),
            'bpe_vocab_size': len(bpe_vocab),
            'ref_avg_token_length': sum(len(t) for t in reference_tokens) / len(reference_tokens),
            'bpe_avg_token_length': sum(len(t) for t in bpe_tokens) / len(bpe_tokens),
            'total_ref_tokens': len(reference_tokens),
            'total_bpe_tokens': len(bpe_tokens),
            'tokenization_accuracy': tokenization_accuracy,
            'tokenization_coverage': tokenization_coverage,
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score,
            'jaccard_similarity': jaccard_similarity,
            'true_positives': true_positives,
            'false_positives': false_positives,
            'false_negatives': false_negatives
        }
        return metrics
    
    print("\nTokenization Comparison:")
    print("-" * 50)
    
for book_name, ref_tokens in reference_tokenizations.items():
        test_book = gutenberg.raw(f"{book_name}.txt")
        bpe_tokens = bpe_tokenizer.encode(test_book)
        
        bpe_tokens = [token.replace('_', '') for token in bpe_tokens]
        
        print(bpe_tokens)
        print(ref_tokens)
                
        metrics = calculate_metrics(ref_tokens, bpe_tokens)
        
        print(f"\nResults for {book_name}:")
        print(f"Reference vocabulary size: {metrics['ref_vocab_size']}")
        print(f"BPE vocabulary size: {metrics['bpe_vocab_size']}")
        print(f"Reference avg token length: {metrics['ref_avg_token_length']:.2f}")
        print(f"BPE avg token length: {metrics['bpe_avg_token_length']:.2f}")
        print(f"Total reference tokens: {metrics['total_ref_tokens']}")
        print(f"Total BPE tokens: {metrics['total_bpe_tokens']}")
        print(f"Tokenization accuracy: {metrics['tokenization_accuracy']:.2f}%")
        print(f"Tokenization coverage: {metrics['tokenization_coverage']:.2f}%")
        print(f"Precision: {metrics['precision']:.2f}")
        print(f"Recall: {metrics['recall']:.2f}")
        print(f"F1 score: {metrics['f1_score']:.2f}")
        print(f"Jaccard similarity: {metrics['jaccard_similarity']:.2f}")
        print(f"True positives: {metrics['true_positives']}")
        print(f"False positives: {metrics['false_positives']}")
        print(f"False negatives: {metrics['false_negatives']}")
